<a href="https://colab.research.google.com/github/Tirta2241720045/PCVK_Ganjil_2024/blob/main/Week12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nama: Tirta Nurrochman Bintang Prawira
# NIM: 2241720045
# Kelas: TI-3A
# No Absen: 27

## Langkah 1 - Import Library

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Langkah 2 - Pra Pengolahan Data

### Langkah 2.1. Pra Pengolahan Data Training

In [2]:
train_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/dataset/Minggu12/Dataset_Daging_Hewan_Ternak/Training',
                                                 target_size = (224, 224),
                                                 batch_size = 48,
                                                 class_mode = 'categorical',
                                                 shuffle = True)

Found 240 images belonging to 3 classes.


### Langkah 2.2. Pra Pengolahan Data Testing

In [3]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/dataset/Minggu12/Dataset_Daging_Hewan_Ternak/Testing',
                                                 target_size = (224, 224),
                                                 batch_size = 48,
                                                 class_mode = 'categorical',
                                                 shuffle = True)

Found 60 images belonging to 3 classes.


## Langkah 3 - Pembuatan Model CNN

In [5]:
num_classes = 3

cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[224, 224, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))  # Menambahkan konvolusi tambahan
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))  # num_classes adalah jumlah kelas dalam dataset Anda
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## Langkah 4 - Fit CNN

In [17]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# 1. Pertama, pastikan generator data sudah benar
print("Training set:", training_set.samples, "samples")
print("Test set:", test_set.samples, "samples")
print("Batch size:", training_set.batch_size)

# 2. Hitung steps dengan lebih aman
steps_per_epoch = training_set.samples // training_set.batch_size
validation_steps = max(1, test_set.samples // test_set.batch_size)  # Minimal 1 step

# 3. Buat callbacks yang lebih sederhana
early_stopping = EarlyStopping(
    monitor='loss',  # Monitor training loss instead of validation
    patience=5,
    restore_best_weights=True,
    verbose=1
)

checkpoint = ModelCheckpoint(
    'best_model.keras',
    monitor='accuracy',  # Monitor accuracy instead of validation accuracy
    save_best_only=True,
    verbose=1
)

# 4. Coba latih model tanpa validation data dulu
history = cnn.fit(
    training_set,
    epochs=25,
    steps_per_epoch=steps_per_epoch,
    callbacks=[early_stopping, checkpoint],
    verbose=1
)

Training set: 240 samples
Test set: 60 samples
Batch size: 48
Epoch 1/25
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.9086 - loss: 0.3640
Epoch 1: accuracy improved from -inf to 0.90417, saving model to best_model.keras
5/5 ━━━━━━━━━━━━━━━━━━━━ 24s 159ms/step - accuracy: 0.9079 - loss: 0.3633
Epoch 2/25

Epoch 2: accuracy did not improve from 0.90417
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/25
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.8658 - loss: 0.3267
Epoch 3: accuracy improved from 0.90417 to 0.91250, saving model to best_model.keras
5/5 ━━━━━━━━━━━━━━━━━━━━ 26s 156ms/step - accuracy: 0.8736 - loss: 0.3129
Epoch 4/25

Epoch 4: accuracy did not improve from 0.91250
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/25
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.9589 - loss: 0.1591
Epoch 5: accuracy improved from 0.91250 to 0.95417, saving model to best_model.keras
5/5 ━━━━━━━━━

# Langkah 5 Monitoring Hasil

In [ ]:
# Tambahan: Evaluasi dan Visualisasi (Optional)
# Evaluasi model
test_loss, test_accuracy = model.evaluate(test_set)
print(f"\nTest Accuracy: {test_accuracy*100:.2f}%")

# Plot training history
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'])

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'])

plt.tight_layout()
plt.show()

2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 3s/step - accuracy: 0.3806 - loss: -18.9890

Test Accuracy: 38.33%


NameError: name 'plt' is not defined

## Langkah 5 - Prediksi dengan 1 Citra

In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt

img_1 = '/content/drive/MyDrive/dataset/Minggu12/Dataset_Daging_Hewan_Ternak/single_prediction/object1.jpg'
img1 = cv.imread(img_1)
prediction = image_predict(img_1)
plt.imshow(cv.cvtColor(img1, cv.COLOR_BGR2RGB))
plt.title(prediction)
plt.axis('off')
plt.show()

NameError: name 'image_predict' is not defined